In [2]:
import pandas as pd
import os
import openai
from IPython.display import display, Markdown

In [2]:
# openai.api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [3]:
# Set openai.api_key to the OPENAI environment variable
# openai.api_key = os.environ["OPENAI"]

In [4]:
df = pd.read_json(path_or_buf="nope_data/nli_corpus.main.jsonl", lines=True)
# df.to_csv("nope_data/data.csv", index=False)

In [5]:
df

,uid,premise,hypothesis,label,metadata
0,1-neg,The cops had him in their headlights. He ran h...,He was standing earlier.,E,{'context1': 'The cops had him in their headli...
1,1,The cops had him in their headlights. He ran h...,He was standing earlier.,E,{'context1': 'The cops had him in their headli...
2,10-neg,For three nights a comet flared through the de...,A red smudge couldn't be seen on the moon before.,E,{'context1': 'For three nights a comet flared ...
3,10,For three nights a comet flared through the de...,A red smudge couldn't be seen on the moon before.,E,{'context1': 'For three nights a comet flared ...
4,1000-neg,""" He just thinks it's so much bull--. He's not...",We are someone.,N,"{'context1': '"" He just thinks it's so much bu..."
...,...,...,...,...,...
2381,991,I've worked obviously suspicious gobblers for ...,They will react in some way.,E,{'context1': 'I've worked obviously suspicious...
2382,993-neg,"And, of course, at that time my dad was dying,...",Something happened.,E,"{'context1': 'And, of course, at that time my ..."
2383,993,"And, of course, at that time my dad was dying,...",Something happened.,E,"{'context1': 'And, of course, at that time my ..."
2384,999-neg,I felt grubby just looking at her. I don't kn...,Elissa has a favorite hobby.,N,{'context1': 'I felt grubby just looking at he...


In [6]:
# dict of format {0: {col1name:val, col2name:val}, 1: {col1name:val, col2name:val} }
df_dict = df.to_dict("index")


In [7]:
# a = df.apply(lambda row: row["premise"], axis=1)
# print(type(a))
# print(a)

In [8]:
# messages = [{"role": "system", "content": "Using the information in the given passage and your background knowledge about how the world works, generate a statement we can infer from it."}]

# shots = 4   # 4 neg
# # for every shot, loop runs twice (one for premise, one for hypothesis)

# for i in range(0,2*shots):
#     my_dict = {}
#     # i is even ie premise 
#     if i % 2 == 0:
#         my_dict["role"] = "user"
#         my_dict["content"] = df_dict[i]["premise"]
#     else:
#         my_dict["role"] = "assisstant"
#         my_dict["content"] = df_dict[i-1]["hypothesis"]
#     print(i)
#     print(my_dict)


In [9]:
prompt_list = [{"role": "system", "content": "Assume the statement after \"Premise\" is true. If the statement after \"Hypothesis\" is also true, output \"entailed\", otherwise output \"not entailed\"."}]
shots = 6 
# 2 pos, 2 neg

In [10]:
def create_fs_msg_list(message_list, shots):
    """append dictionary of user and assisstant messages for few-shot technique"""

    for i in range(0,2*shots):
        # for every shot, loop runs twice (one for user, one for assisstant)
        my_dict = {}

        if i % 2 == 0:
            prem = df_dict[i/2]["premise"]
            hyp = df_dict[i/2]["hypothesis"]
            my_dict["role"] = "user"
            my_dict["content"] = f"Premise: {prem} \nHypothesis: {hyp}"
        else:
            label = "entailed" if df_dict[(i-1)/2]["label"] == "E" else "not entailed"
            my_dict["role"] = "assisstant"
            my_dict["content"] = label
            
        print(my_dict)
        message_list.append(my_dict)
        # print(message_list)
    return message_list

In [11]:
# def create_fs_msg_list(message_list, shots):
#     """append dictionary of user and assisstant messages for few-shot technique"""

#     for i in range(0,2*shots):
#         # for every shot, loop runs twice (one for premise, one for hypothesis)
#         my_dict = {}
#         # i is even ie premise 
#         if i % 2 == 0:
#             my_dict["role"] = "user"
#             my_dict["content"] = df_dict[i/2]["premise"]
#         else:
#             my_dict["role"] = "assisstant"
#             my_dict["content"] = df_dict[(i-1)/2]["hypothesis"]
#         print(my_dict)
#         message_list.append(my_dict)
#     return message_list

In [12]:
message_list_fs = create_fs_msg_list(prompt_list, shots)


{'role': 'user', 'content': "Premise: The cops had him in their headlights. He ran hard and fast, fiercely pumping his legs, his arms, but they gained on him quickly, swerving in front of him to block his way. Winded, aching, he didn't fall on his knees in the street. \nHypothesis: He was standing earlier."}
{'role': 'assisstant', 'content': 'entailed'}
{'role': 'user', 'content': 'Premise: The cops had him in their headlights. He ran hard and fast, fiercely pumping his legs, his arms, but they gained on him quickly, swerving in front of him to block his way. Winded, aching, he fell on his knees in the street. \nHypothesis: He was standing earlier.'}
{'role': 'assisstant', 'content': 'entailed'}
{'role': 'user', 'content': "Premise: For three nights a comet flared through the desert sky. The winds hooted like owls. A red smudge didn't appear on the moon. \nHypothesis: A red smudge couldn't be seen on the moon before."}
{'role': 'assisstant', 'content': 'entailed'}
{'role': 'user', 'con

In [13]:
def make_prompt(example, message_list):
    prompt = list(message_list)
    prompt.append({"role": "user", "content": example})
    return prompt

In [ ]:
def get_response()

In [20]:
def give_gpt_msg(message_list, shots):
    """give the premise for which we want LM to generate a presupposition"""

    response_dict = {}
    for i in range(len(df_dict)):

        if i <= shots-1:
            continue
        
        prompt = make_prompt(df_dict[i]["premise"], message_list)
        # passing the message list to chat gpt api
        response = openai.ChatCompletion.create(model="gpt-3.5-turbo", 
                        messages = prompt)
        response_message = response["choices"][0]["message"]
        response_dict[i] = response_message
        # # print(response)
        print(prompt)
        # if i > 7:
        #     # print(prompt)
        #     break
    return message_list, response_dict
            

In [21]:
message_list_final, response_dict = give_gpt_msg(message_list_fs, shots)

[{'role': 'system', 'content': 'Assume the statement after "Premise" is true. If the statement after "Hypothesis" is also true, output "entailed", otherwise output "not entailed".'}, {'role': 'user', 'content': "Premise: The cops had him in their headlights. He ran hard and fast, fiercely pumping his legs, his arms, but they gained on him quickly, swerving in front of him to block his way. Winded, aching, he didn't fall on his knees in the street. \nHypothesis: He was standing earlier."}, {'role': 'assisstant', 'content': 'entailed'}, {'role': 'user', 'content': 'Premise: The cops had him in their headlights. He ran hard and fast, fiercely pumping his legs, his arms, but they gained on him quickly, swerving in front of him to block his way. Winded, aching, he fell on his knees in the street. \nHypothesis: He was standing earlier.'}, {'role': 'assisstant', 'content': 'entailed'}, {'role': 'user', 'content': "Premise: For three nights a comet flared through the desert sky. The winds hoot

[{'role': 'system',
  'content': 'Assume the statement after "Premise" is true. If the statement after "Hypothesis" is also true, output "entailed", otherwise output "not entailed".'},
 {'role': 'user',
  'content': "Premise: The cops had him in their headlights. He ran hard and fast, fiercely pumping his legs, his arms, but they gained on him quickly, swerving in front of him to block his way. Winded, aching, he didn't fall on his knees in the street. \nHypothesis: He was standing earlier."},
 {'role': 'assisstant', 'content': 'entailed'},
 {'role': 'user',
  'content': 'Premise: The cops had him in their headlights. He ran hard and fast, fiercely pumping his legs, his arms, but they gained on him quickly, swerving in front of him to block his way. Winded, aching, he fell on his knees in the street. \nHypothesis: He was standing earlier.'},
 {'role': 'assisstant', 'content': 'entailed'},
 {'role': 'user',
  'content': "Premise: For three nights a comet flared through the desert sky. T

In [ ]:
# Add new column to df
new_column = pd.Series(response_dict)
df["GPT Response"] = new_column

In [ ]:
def get_accuracy_score(shots, df):
    start_row = 2  # Specify the row number to start the comparison from

# Select a subset of rows from the specified row number onwards
subset_df = df.loc[start_row:]

# Compare the values in the selected subset of rows
subset_df['Comparison'] = subset_df['Column1'] > subset_df['Column2']

In [6]:
# data = {0: [3, 2, 1, 0], 1: ['a', 'b', 'c', 'd']}
# df1 = pd.DataFrame.from_dict(data, orient='index')
# df1

,0,1,2,3
0,3,2,1,0
1,a,b,c,d


In [8]:
# test_dict = {1: "n"}
# new_column = pd.Series(test_dict)
# df1['new col'] = new_column

In [16]:
# completion = openai.ChatCompletion.create(
#   model="gpt-3.5-turbo", 
#   messags = [{"role": "system", "content" : "You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible.\nKnowledge cutoff: 2021-09-01\nCurrent date: 2023-03-02"},
# {"role": "user", "content" : "How are you?"},
# {"role": "assistant", "content" : "I am doing well"},
# {"role": "user", "content" : "What is the mission of the company OpenAI?"}]
# )
# #print(completion)
# print(completion)

In [ ]:
# completion = openai.ChatCompletion.create(
#   model="gpt-3.5-turbo", 
#   messags = create_msg_list()
# )
# #print(completion)
# print(completion)